In [53]:
import re
import numpy as np

with open("data.txt") as file:
    data = file.read()

In [54]:
pipe_map = []
length_map = []

for line in  data.splitlines():
    pipe_map.append(list(line))
    length_map.append([0 for _ in range(len(line))])

pipe_dict = {
    "|": [(1, 0), (-1, 0)],
    "-": [(0, 1), (0, -1)],
    "L": [(-1, 0), (0, 1)],
    "J": [(-1, 0), (0, -1)],
    "7": [(1, 0), (0, -1)],
    "F": [(1, 0), (0, 1)],
}

start_cord = [
    (y, x)
    for y, row in enumerate(pipe_map)
    for x, cell in enumerate(row)
    if cell == "S"
][0]

start_neighbours = [
    (start_cord[0] + 1, start_cord[1]),
    (start_cord[0] - 1, start_cord[1]),
    (start_cord[0], start_cord[1] + 1),
    (start_cord[0], start_cord[1] - 1),
]

possible_neighbours = []

for cord in start_neighbours:
    if cord[0] >= 0 and cord[1] >= 0:
        pipe = pipe_map[cord[0]][cord[1]]
        if pipe != ".":
            pipe_dirs = pipe_dict[pipe]
            possible_cords = [
                (cord[0] + pipe_dirs[0][0], cord[1] + pipe_dirs[0][1]),
                (cord[0] + pipe_dirs[1][0], cord[1] + pipe_dirs[1][1]),
            ]
            if start_cord in possible_cords:
                possible_neighbours.append(cord)


def generate_lenghts(last_position, current_position):

    count_map = length_map.copy()
    steps = 1
    path = []

    while current_position != start_cord:

        count_map[current_position[0]][current_position[1]] = steps

        pipe = pipe_map[current_position[0]][current_position[1]]
        #print(current_position, pipe)
        
        pipe_dirs = pipe_dict[pipe]
        possible_cords = [
            (current_position[0] + pipe_dirs[0][0], current_position[1] + pipe_dirs[0][1]),
            (current_position[0] + pipe_dirs[1][0], current_position[1] + pipe_dirs[1][1]),
        ]

        possible_cords.remove(last_position)
        path.append(last_position)
        last_position = current_position
        current_position = possible_cords[0]
        steps += 1
    
    path.append(last_position)
    return np.array(count_map), path

path_1, p1 = generate_lenghts(start_cord, possible_neighbours[0])
path_2, _ = generate_lenghts(start_cord, possible_neighbours[1])

path_1[(path_2 == path_1) & (path_1 != 0) & (path_2 != 0)]

array([70])

In [55]:
inside = []
outside = []

path_1[start_cord] = 100
last_cord = p1[-1]

all_cords = {(y, x) for y, row in enumerate(pipe_map) for x, _ in enumerate(row)}

left_dir = {
    (1, 0): (0, 1),
    (-1, 0): (0, -1),
    (0, 1): (-1, 0),
    (0, -1): (1, 0),
}

right_dir = {
    (1, 0): (0, -1),
    (-1, 0): (0, 1),
    (0, 1): (1, 0),
    (0, -1): (-1, 0),
}

for current_cord in p1:
    pipe = pipe_map[current_cord[0]][current_cord[1]]

    heading = np.array(current_cord) - np.array(last_cord)
    r_dir = right_dir[tuple(heading)]
    l_dir = left_dir[tuple(heading)]

    if ((left_y := l_dir[0] + current_cord[0]),(left_x := l_dir[1] + current_cord[1])) in all_cords:
        l_dir_pipe = pipe_map[left_y][left_x]
        if l_dir_pipe == ".":
            inside.append((left_y, left_x))

    if ((rigth_y := r_dir[0] + current_cord[0]), (right_x := r_dir[1] + current_cord[1])) in all_cords:
        r_dir_pipe = pipe_map[rigth_y][right_x]

        if r_dir_pipe == ".":
            outside.append((rigth_y, right_x))

    last_cord = current_cord

In [61]:
temp_map = [a.copy() for a in pipe_map]

for d in outside:
    temp_map[d[0]][d[1]] = "I"

for l in temp_map:
    for a in l:
        print(a, end="")
    print(end='\n')

print()
for l in pipe_map:
    for a in l:
        print(a, end="")
    print(end='\n')

.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJIL-7..
L--J.L7I.ILJS7F-7L7.
....F-JI.F7FJ|L7L7L7
....L7IF7||L7|IL7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...

.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...


In [57]:
set(outside)

{(3, 14), (4, 7), (4, 9), (5, 7), (6, 6), (6, 14)}

In [58]:
len(set(outside))

6

In [59]:
len(set(inside))

24